# Send Gmail with Attachments

Original code from https://gist.github.com/rdempsey/22afd43f8d777b78ef22

In [1]:
import os
import sys
import smtplib
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from send_gmail_conf import *

In [2]:
def send_gmail(gmail_address=YOUR_GMAIL_ADDRESS,
         gmail_password=YOUR_GMAIL_PASSWORD,
         recipient_list=RECIPIENT_LIST,
         subject=SUBJECT,
         text=TEXT,
         attachment_list=ATTACHMENT_LIST):
    """
    Uses gmail.com to send an email message, optionally with a list of attachments, to a list of recipients.
    
    Arguments for this function are imported from send_gmail_conf.py which must exist in the same directory.
    
    send_gmail_conf.py is in this format:
    
    YOUR_GMAIL_ADDRESS = 'user@gmail.com'
    YOUR_GMAIL_PASSWORD = 'password'
    RECIPIENT_LIST = 'user@guam.net,user2@somewhere.com'
    SUBJECT = 'Test'
    TEXT = 'Hi there'
    ATTACHMENT_LIST = ['1Mcolors.png', 'afile.doc']
    
    Use the following line of code to import:
    
    from send_gmail_conf import *    
    """
    # Create the enclosing (outer) message
    outer = MIMEMultipart()
    outer['Subject'] = subject
    outer['To'] = ', '.join(recipient_list)
    outer['From'] = gmail_address
    outer.preamble = 'You will not see this in a MIME-aware mail reader.\n'
    #text = "EMAIL MESSAGE"
    outer.attach(MIMEText(text, 'plain')) # or 'html'

    # Add the attachments to the message
    for file in attachment_list:
        try:
            with open(file, 'rb') as fp:
                msg = MIMEBase('application', "octet-stream")
                msg.set_payload(fp.read())
            encoders.encode_base64(msg)
            msg.add_header('Content-Disposition', 'attachment', filename=os.path.basename(file))
            outer.attach(msg)
        except:
            print("Unable to open one of the attachments. Error: ", sys.exc_info()[0])
            raise

    composed = outer.as_string()

    # Send the email
    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as s:
            s.ehlo()
            s.starttls()
            s.ehlo()
            s.login(gmail_address, gmail_password)
            s.sendmail(gmail_address, recipient_list, composed)
            s.close()
        print("Email sent!")
    except:
        print("Unable to send the email. Error: ", sys.exc_info()[0])
        raise

# send_gmail()

Email sent!
